In [ ]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline

In [ ]:
df = pd.read_csv('seattle-weather.csv')
df.head()

,date,precipitation,temp_max,temp_min,wind,weather
0,2012-01-01,0.0,12.8,5.0,4.7,drizzle
1,2012-01-02,10.9,10.6,2.8,4.5,rain
2,2012-01-03,0.8,11.7,7.2,2.3,rain
3,2012-01-04,20.3,12.2,5.6,4.7,rain
4,2012-01-05,1.3,8.9,2.8,6.1,rain


In [ ]:

# 1. Drop kolom tidak relevan
X = df.drop(['weather', 'date'], axis=1)
y = df['weather']

# Encode label target
le = LabelEncoder()
y = le.fit_transform(y)

# 2. Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Pipeline dengan scaling dan SVC
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', SVC(probability=True))
])

# 4. Parameter grid
param_grid = {
    'svc__C': [0.1, 1, 10],
    'svc__kernel': ['linear', 'rbf'],
    'svc__gamma': ['scale', 0.1, 1],
    'svc__class_weight': [None, 'balanced']
}

# 5. GridSearchCV
model = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')
model.fit(X_train, y_train)

# 6. Evaluasi pada test set
print("Parameter terbaik:", model.best_params_)
print("Akurasi pada test set:", model.score(X_test, y_test))

Parameter terbaik: {'svc__C': 10, 'svc__class_weight': None, 'svc__gamma': 'scale', 'svc__kernel': 'linear'}
Akurasi pada test set: 0.8293515358361775


In [ ]:
def predict_weather():
    print("Masukkan parameter cuaca untuk prediksi:")
    precipitation = float(input("Precipitation (mm): "))
    temp_max = float(input("Temp Max (°C): "))
    temp_min = float(input("Temp Min (°C): "))
    wind = float(input("Wind speed (km/h): "))

    # Buat DataFrame dari input user
    user_input = pd.DataFrame([[precipitation, temp_max, temp_min, wind]],
                             columns=['precipitation', 'temp_max', 'temp_min', 'wind'])

    # Prediksi
    prediction = model.predict(user_input)
    proba = model.predict_proba(user_input)

    # Decode label
    weather_predicted = le.inverse_transform(prediction)[0]
    classes = le.classes_

    print("\nHasil Prediksi:")
    print(f"Cuaca: {weather_predicted}")
    print("Probabilitas:")
    for cls, prob in zip(classes, proba[0]):
        print(f"- {cls}: {prob*100:.2f}%")

predict_weather()

Masukkan parameter cuaca untuk prediksi:
Precipitation (mm): 10.9
Temp Max (°C): 10.6
Temp Min (°C): 1.8
Wind speed (km/h): 4.5

Hasil Prediksi:
Cuaca: rain
Probabilitas:
- drizzle: 0.02%
- fog: 0.04%
- rain: 92.76%
- snow: 7.18%
- sun: 0.00%
